In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [6]:

services = [["https://cazton.com/consulting/artificial-intelligence/ai-express-poc",
 "https://cazton.com/consulting/artificial-intelligence/ai-agents",
 "https://cazton.com/consulting/artificial-intelligence/openai",
 "https://cazton.com/consulting/artificial-intelligence/openai-case-studies-collection",
 "https://cazton.com/consulting/big-data-development/apache-spark"]]

loader = WebBaseLoader(["https://cazton.com/consulting/artificial-intelligence/ai-express-poc",
 "https://cazton.com/consulting/artificial-intelligence/ai-agents",
 "https://cazton.com/consulting/artificial-intelligence/openai",
 "https://cazton.com/consulting/artificial-intelligence/openai-case-studies-collection",
 "https://cazton.com/consulting/big-data-development/apache-spark"])

In [60]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,)

In [ ]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceHubEmbeddings
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key="----hf-key----", model_name="sentence-transformers/all-MiniLM-l6-v2"
)

In [88]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
# Prompt template
# template = """Answer the question based only on the following context, 
# which can include text and tables::
# {context}
# Question: {question}
# """
template = """Frame the 10 questions with its answers based on the {question} using the context given below in triple backticks. If the
        question cannot be framed using the information provided answer
        with "I don't know".

        Context: Let's think of the situation. You are given text now after reading text your task is 
        to think of ideal question with its answers for that text. 

        Instruction: 
        DO NOT INCLUDE any personal question about a person,place etc.

        text: ```{text}```

        Do Return the question answers in following format:-

        {{questions : ["quest 1", "quest 2"],
          answers : ["ans1", "ans2"]}}
        
        """

prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(temperature=0.5, openai_api_key="---open-ai--key")



In [62]:
import pandas as pd

In [64]:
df = pd.DataFrame(columns = ["questions", "answers", "service"])

In [94]:
import requests
from bs4 import BeautifulSoup

# Replace 'your_url' with the actual URL of the webpage you want to scrape

url = "https://cazton.com/consulting/web-development"


# Send a GET request to the URL
response = requests.get(url)

# Parse the HTML content of the page using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find the particular element you're interested in (replace 'your_element' with the actual tag and class or id)
target_element = soup.find_all("a" , class_ = "btn radius-50 btn-gray-transparent btn-animated")
services = []
for el in target_element:
    # Check if the element is found
    if el:
        # Get the value of the 'href' attribute
        href_value = el.get('href')
        if href_value not in services:
            href_value = "https://cazton.com/" + href_value
            services.append(href_value)

        # Print or use the href value as needed
        print("Href Value:", href_value)
    else:
        print("Element not found")


Href Value: https://cazton.com//consulting/web-development/dotnet7
Href Value: https://cazton.com//consulting/web-development/dotnet8
Href Value: https://cazton.com//consulting/web-development/java
Href Value: https://cazton.com//consulting/web-development/dotnet6
Href Value: https://cazton.com//consulting/web-development/dotnet5
Href Value: https://cazton.com//consulting/web-development/dotnet-core
Href Value: https://cazton.com//consulting/web-development/dotnet
Href Value: https://cazton.com//consulting/web-development/web-development
Href Value: https://cazton.com//consulting/web-development/efcore
Href Value: https://cazton.com//consulting/web-development/blazor
Href Value: https://cazton.com//consulting/web-development/angular
Href Value: https://cazton.com//consulting/microsoft/microsoft-consulting
Href Value: https://cazton.com//consulting/web-development/react
Href Value: https://cazton.com//consulting/devops/docker
Href Value: https://cazton.com//consulting/web-development/no

In [95]:
services = services[8:]

for service in services:
    loader = WebBaseLoader(service)
    data = loader.load()
    texts = text_splitter.split_documents(data)
    db = FAISS.from_documents(texts, embeddings)
    retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 8})
    # RAG pipeline
    chain = (
        {"text": retriever, "question": RunnablePassthrough()}
        | prompt
        | model
        | StrOutputParser()
    )

    # Name of the service
    service = service.split("/")[-1]
    result = chain.invoke(input = service)
    try:
        result = eval(result)
        result.update({"service" : service})
        # temp = pd.DataFrame({"questions" : eval(result["questions"]), "answers" : eval(result["answers"]), "service" : service})
        temp = pd.DataFrame(result)
        print(temp)
        df = pd.concat([df, temp], axis= 0)
    except:
        pass

                                           questions  \
0                                    What is .NET 7?   
1                          When was .NET 7 released?   
2                   What can you build using .NET 7?   
3  How many performance-impacting pull requests d...   
4  What is the support timeline for .NET MAUI in ...   
5  What are some new features and improvements in...   
6  What technologies does Cazton support in the ....   
7  How many community members have contributed to...   
8            What are the main principles of .NET 7?   
9  What are some new features and improvements in...   

                                             answers  service  
0  .NET 7 is a free, cross-platform, development ...  dotnet7  
1             .NET 7 was released in November, 2022.  dotnet7  
2  .NET 7 can be used to build web, mobile, deskt...  dotnet7  
3  .NET 7 had more than 7000 performance-impactin...  dotnet7  
4  The support for .NET MAUI in .NET 7 will be av...  dotnet7  

InvalidRequestError: This model's maximum context length is 4097 tokens. However, your messages resulted in 7469 tokens. Please reduce the length of the messages.

In [ ]:
topic = url.split("/")[-1]
df.to_csv(f"qa-pair-{topic}.csv", index = False)